# MNIST

### MNIST 데이터 받기

In [1]:
import os
import sys
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("data/", one_hot=True)
train_data = mnist.train.images
train_label = mnist.train.labels
test_data = mnist.test.images
test_label = mnist.test.labels

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


### MNIST 데이터 확인하기

In [2]:
# size of MNIST
print(train_data.shape)
print(train_label.shape)
print(test_data.shape)
print(test_label.shape)

(55000, 784)
(55000, 10)
(10000, 784)
(10000, 10)


### Data Pre-processing (데이터 전처리)


# Implementation (1)


## Loss function (손실 함수) : Cross Entropy

# <center> \\( L(y_i, f(x_i; W)) = -\frac{1}{n}\sum_{i=1}^{n}\sum_{k=1} y_{i,j} log(f(x_i)_k)\\)</center>


#### get_cross_entropy_loss 함수의 내용을 완성하세요.
#### (Hint : (1) tf.reduce_mean(), tf.reduce_sum(), tf.log() (2) Tensor dimension에 유의 (3) log 함수 사용 시 epsilon 사용하세요.)

In [3]:
def get_cross_entropy_loss(y_true, y_hat, epsilon=1e-8):
    """
    compute cross entropy

    Args:
        y_true: true label
        y_hat: predicted label
        epsilon: small value to prevent NaN in log

    Returns:
        cross entropy loss
    """
    with tf.name_scope('cross_entropy'):
        loss = -tf.reduce_mean(tf.reduce_sum((y_true*tf.log(y_hat)), axis=1))
    return loss
        ###################################################################
        #                    Implementation 1                             #
        ###################################################################
y_true = tf.placeholder(tf.float32, [None,10])
y_hat=tf.placeholder(tf.float32,[None,10])
get_cross_entropy_loss(y_true, y_hat)

def get_accuracy(y_true, y_hat):
    """
    compute cross entropy

    Args:
        y_true: true label
        y_hat: predicted label
        epsilon: small value to prevent NaN in log

    Returns:
        cross entropy loss
    """
    with tf.name_scope('accuracy'):
        # Compare the highest indices between the predicted label and the true label
        correct_prediction = tf.equal(tf.argmax(y_hat, 1), tf.argmax(y_true, 1), name='correct_prediction')
        # Compute accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')
    return accuracy

## Hyper-parameter (하이퍼 파라미터)

In [12]:
# Set hypyerparameters
learning_rate = 0.001
max_iter = 2000
batch_size = 100

In [5]:
inputs = tf.placeholder(tf.float32, [None,784])
inputs.get_shape().as_list()

[None, 784]

# Implementation (2)

## Linear Classifier (선형 분류기)

## <center> \\( f(x) = W^Tx+b \\)</center>

### linear function을 완성하세요.
### (Hint : (1) weight, bias 선언 (2) tf.get_variable()의 initializer  (3) tf.matmul())

In [8]:
def fc(name, out_dim, inputs):
    """
    Args :
        Inputs : Input tensor
        out_dim : output dimension
        
    Returns:
        inputs * weight + bias
    """
    with tf.variable_scope(name):
        shp = inputs.get_shape().as_list()[-1]
        initials = tf.truncated_normal([shp ,out_dim], stddev=0.01)
        weight = tf.get_variable('w', initializer = initials)

        initials = tf.constant(1.0, shape=[out_dim])
        bias = tf.get_variable('b', initializer = initials)

        result = tf.matmul(inputs, weight)+bias
        return result
    
    
    with tf.name_scope('linear_classifier'):
        
        return y_hat
        ###################################################################
        #                    Implementation 2                             #
        ###################################################################
        



# Implementation (3)

## Model Setting

### 1. Training data 및 Test data의 각각의 image를 한 vector로 만들어서 train_data, test_data에 각각 저장하세요.
#### Hint) 데이터 차원.
### 2. Dataset로부터 받은 데이터(Image, label)를 담을 변수를 각각 x 및 y_true에 선언하세요.
#### Hint) tf.placeholder
### 3. Implementation (2)에서 구현한 linear classifier 함수값과 softmax 함수를 통한 prediction 값을 y_hat에 저장하세요.
#### Hint) tf.nn.softmax
### 4. 3으로부터 얻은 결과를 통해 Implementation (1)에서 구현한 loss function을 통해 얻은 loss를 cross_entropy에 저장하세요.

In [13]:
tf.reset_default_graph()
# Flatten data
###################################################################
#                    Implementation 3-1                           #
###################################################################
# train_data = None
# test_data = None

###################################################################
#                    Implementation 3-2                           #
###################################################################
x = tf.placeholder(tf.float32, [None, 784]);
y_true = tf.placeholder(tf.float32, [None, 10]);

###################################################################
#                    Implementation 3-3                          #
###################################################################

h1 = fc('layer1',512,x)
h1 = tf.nn.relu(h1)

h2 = fc('layer2', 64, h1)
h2 = tf.nn.relu(h2)

y_logits = fc('layer3', 10, h2)
y_hat = tf.nn.softmax(y_logits)

###################################################################
#                    Implementation 3-4                           #
###################################################################
cross_entropy = get_cross_entropy_loss(y_true, y_hat);

# Calculate accuracy
accuracy = get_accuracy(y_true, y_hat)
# Make gradient descent op
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

# Make op to initialize declared variable
init = tf.global_variables_initializer()

with tf.Session() as sess:
    # Initialize variables
    sess.run(init)
    
    # Training
    for step in range(max_iter):
        # Get batch data and label
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # train the network and calculate cross entropy
        #_, loss = sess.run([train_step, cross_entropy], feed_dict={x: batch_x, y_true: batch_y})
        _= sess.run(train_step, feed_dict={x: batch_x, y_true: batch_y})
        loss = sess.run(cross_entropy, feed_dict={x: batch_x, y_true: batch_y})
        # calcualte accuracy
        acc = sess.run(accuracy, feed_dict={x: test_data, y_true: test_label})
        # print loss (cross entropy) and accuracy at every 10th step
        if (step + 1) % 10 == 0:
            print("{}th iteration, loss: {:.4f}, test accuracy: {:.4f}".format(step + 1, loss, acc))

10th iteration, loss: 2.2464, test accuracy: 0.3471
20th iteration, loss: 2.0376, test accuracy: 0.3794
30th iteration, loss: 1.4382, test accuracy: 0.5660
40th iteration, loss: 0.9121, test accuracy: 0.7259
50th iteration, loss: 0.6772, test accuracy: 0.7500
60th iteration, loss: 0.5367, test accuracy: 0.7932
70th iteration, loss: 0.6320, test accuracy: 0.8194
80th iteration, loss: 0.6286, test accuracy: 0.8416
90th iteration, loss: 0.5741, test accuracy: 0.8543
100th iteration, loss: 0.4148, test accuracy: 0.8694
110th iteration, loss: 0.5145, test accuracy: 0.8738
120th iteration, loss: 0.5150, test accuracy: 0.8763
130th iteration, loss: 0.5236, test accuracy: 0.8736
140th iteration, loss: 0.5587, test accuracy: 0.8825
150th iteration, loss: 0.3649, test accuracy: 0.8929
160th iteration, loss: 0.4044, test accuracy: 0.8920
170th iteration, loss: 0.3988, test accuracy: 0.8938
180th iteration, loss: 0.3654, test accuracy: 0.9003
190th iteration, loss: 0.3965, test accuracy: 0.8929
20

1550th iteration, loss: 0.1438, test accuracy: 0.9674
1560th iteration, loss: 0.0475, test accuracy: 0.9684
1570th iteration, loss: 0.0795, test accuracy: 0.9678
1580th iteration, loss: 0.0468, test accuracy: 0.9692
1590th iteration, loss: 0.1105, test accuracy: 0.9672
1600th iteration, loss: 0.0527, test accuracy: 0.9658
1610th iteration, loss: 0.0824, test accuracy: 0.9684
1620th iteration, loss: 0.0347, test accuracy: 0.9677
1630th iteration, loss: 0.0647, test accuracy: 0.9680
1640th iteration, loss: 0.1333, test accuracy: 0.9686
1650th iteration, loss: 0.0570, test accuracy: 0.9682
1660th iteration, loss: 0.0807, test accuracy: 0.9698
1670th iteration, loss: 0.0829, test accuracy: 0.9701
1680th iteration, loss: 0.0807, test accuracy: 0.9712
1690th iteration, loss: 0.0494, test accuracy: 0.9710
1700th iteration, loss: 0.0344, test accuracy: 0.9711
1710th iteration, loss: 0.0561, test accuracy: 0.9726
1720th iteration, loss: 0.0703, test accuracy: 0.9671
1730th iteration, loss: 0.04